In [28]:
# This reloads the extension if already loaded - Everytime you update the .py files, re-run this cell!
%reload_ext autoreload
# Automatically reloads modules before executing code OR makes Jupyter reload your .py files whenever you run a cell.
%autoreload 2

In [4]:
# Importing all of the required python packages
import sys
import os

In [30]:
# Setting up the folder paths.
sys.path.append('./lib')

# Import resuable code (scripts) within .py files.
from data_loader import load_csv, save_df_to_csv
from data_wrangle import add_cols, remove_cols, remove_rows, df_formater, remove_nan_cols, remove_nan_rows, col_name_changer, df_split_state_city, remove_leading_wspace, df_split

Section: Loading and converting data sets into DataFrames.

In [ ]:
df_indicators = load_csv('./data/raw/U.S._Chronic_Disease_Indicators.csv')
# print(df.head())

In [9]:
df_census = load_csv('./data/raw/US_Census_Data_2022_v04_transpose.csv')
# display(df3.head())

Section: Data Wrangling

Census DataFrame: df_census
1. Adding the 'State' column to the dataframe.
2. Adding the correct state name to the newly created State column.
3. Removing blank columns.
4. Removing blank rows.
5. Split DataFrame into 2 separate DataFrames, 1 with state and 1 with city, state; City, State Dataset no long need.
6. Split state DataFrame into 2 separate Dataset on State for  Estimate and Margin of Error.
7. Save both DataFrames into csv files in /data/processed folder.

In [11]:
col_names = ['State']
df_census_temp0 = add_cols(df_census, col_names)
display(df_census_temp0)    # For debugging only - comment out when not needed.

,State,Label (Grouping),TOTAL NUMBER OF RACES REPORTED,TOTAL NUMBER OF RACES REPORTED!!Total population,TOTAL NUMBER OF RACES REPORTED!!Total population!!One race,TOTAL NUMBER OF RACES REPORTED!!Total population!!Two races,TOTAL NUMBER OF RACES REPORTED!!Total population!!Three races,TOTAL NUMBER OF RACES REPORTED!!Total population!!Four or more races,SEX AND AGE,SEX AND AGE!!Total population,...,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed),GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!Less than 30 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!30 percent or more,GROSS RENT,GROSS RENT!!Occupied units paying rent,GROSS RENT!!Occupied units paying rent!!Median gross rent (dollars),COMPUTERS AND INTERNET USE,COMPUTERS AND INTERNET USE!!Total households,COMPUTERS AND INTERNET USE!!Total households!!With a computer,COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription
0,,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,Total population,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,Estimate,NaN,"333,287,562",87.50%,11.80%,0.70%,0.10%,NaN,"333,287,562",...,"42,007,061",48.10%,51.90%,NaN,"42,971,061","1,300",NaN,"129,870,928",95.70%,91.00%
3,,Margin of Error,NaN,*****,±0.1,±0.1,±0.1,±0.1,NaN,*****,...,"±160,112",±0.2,±0.2,NaN,"±162,515",±3,NaN,"±136,261",±0.1,±0.1
4,,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,,Margin of Error,NaN,*****,±0.8,±0.9,±0.2,±0.1,NaN,*****,...,"±3,374",±3.0,±3.0,NaN,"±3,410",±36,NaN,"±1,877",±0.6,±0.8
780,,"Milwaukee County, Wisconsin",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
781,,Total population,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
782,,Estimate,NaN,"918,661",85.50%,13.90%,0.60%,0.10%,NaN,"918,661",...,"188,551",49.10%,50.90%,NaN,"192,905","1,012",NaN,"388,440",94.40%,90.40%


In [13]:
df_census_temp1 = df_formater(df_census_temp0)
display(df_census_temp1)    # For debugging only - comment out when not needed.

,State,Label (Grouping),TOTAL NUMBER OF RACES REPORTED,TOTAL NUMBER OF RACES REPORTED!!Total population,TOTAL NUMBER OF RACES REPORTED!!Total population!!One race,TOTAL NUMBER OF RACES REPORTED!!Total population!!Two races,TOTAL NUMBER OF RACES REPORTED!!Total population!!Three races,TOTAL NUMBER OF RACES REPORTED!!Total population!!Four or more races,SEX AND AGE,SEX AND AGE!!Total population,...,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed),GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!Less than 30 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!30 percent or more,GROSS RENT,GROSS RENT!!Occupied units paying rent,GROSS RENT!!Occupied units paying rent!!Median gross rent (dollars),COMPUTERS AND INTERNET USE,COMPUTERS AND INTERNET USE!!Total households,COMPUTERS AND INTERNET USE!!Total households!!With a computer,COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription
0,United States,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,United States,Total population,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,United States,Estimate,NaN,"333,287,562",87.50%,11.80%,0.70%,0.10%,NaN,"333,287,562",...,"42,007,061",48.10%,51.90%,NaN,"42,971,061","1,300",NaN,"129,870,928",95.70%,91.00%
3,United States,Margin of Error,NaN,*****,±0.1,±0.1,±0.1,±0.1,NaN,*****,...,"±160,112",±0.2,±0.2,NaN,"±162,515",±3,NaN,"±136,261",±0.1,±0.1
4,Alabama,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,"Dane County, Wisconsin",Margin of Error,NaN,*****,±0.8,±0.9,±0.2,±0.1,NaN,*****,...,"±3,374",±3.0,±3.0,NaN,"±3,410",±36,NaN,"±1,877",±0.6,±0.8
780,"Milwaukee County, Wisconsin","Milwaukee County, Wisconsin",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
781,"Milwaukee County, Wisconsin",Total population,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
782,"Milwaukee County, Wisconsin",Estimate,NaN,"918,661",85.50%,13.90%,0.60%,0.10%,NaN,"918,661",...,"188,551",49.10%,50.90%,NaN,"192,905","1,012",NaN,"388,440",94.40%,90.40%


In [15]:
df_census_temp2 = remove_nan_cols(df_census_temp1)
display(df_census_temp2)    # For debugging only - comment out when not needed.

,State,Label (Grouping),TOTAL NUMBER OF RACES REPORTED!!Total population,TOTAL NUMBER OF RACES REPORTED!!Total population!!One race,TOTAL NUMBER OF RACES REPORTED!!Total population!!Two races,TOTAL NUMBER OF RACES REPORTED!!Total population!!Three races,TOTAL NUMBER OF RACES REPORTED!!Total population!!Four or more races,SEX AND AGE!!Total population,SEX AND AGE!!Total population!!Male,SEX AND AGE!!Total population!!Female,...,OWNER CHARACTERISTICS!!Owner-occupied housing units!!Median selected monthly owner costs with a mortgage (dollars),OWNER CHARACTERISTICS!!Owner-occupied housing units!!Median selected monthly owner costs without a mortgage (dollars),GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed),GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!Less than 30 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!30 percent or more,GROSS RENT!!Occupied units paying rent,GROSS RENT!!Occupied units paying rent!!Median gross rent (dollars),COMPUTERS AND INTERNET USE!!Total households,COMPUTERS AND INTERNET USE!!Total households!!With a computer,COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription
0,United States,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,United States,Total population,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,United States,Estimate,"333,287,562",87.50%,11.80%,0.70%,0.10%,"333,287,562",49.60%,50.40%,...,"1,775",585,"42,007,061",48.10%,51.90%,"42,971,061","1,300","129,870,928",95.70%,91.00%
3,United States,Margin of Error,*****,±0.1,±0.1,±0.1,±0.1,*****,±0.1,±0.1,...,±3,±1,"±160,112",±0.2,±0.2,"±162,515",±3,"±136,261",±0.1,±0.1
4,Alabama,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,"Dane County, Wisconsin",Margin of Error,*****,±0.8,±0.9,±0.2,±0.1,*****,±0.1,±0.1,...,±50,±21,"±3,374",±3.0,±3.0,"±3,410",±36,"±1,877",±0.6,±0.8
780,"Milwaukee County, Wisconsin","Milwaukee County, Wisconsin",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
781,"Milwaukee County, Wisconsin",Total population,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
782,"Milwaukee County, Wisconsin",Estimate,"918,661",85.50%,13.90%,0.60%,0.10%,"918,661",48.70%,51.30%,...,"1,544",684,"188,551",49.10%,50.90%,"192,905","1,012","388,440",94.40%,90.40%


In [17]:
df_census_temp3 = remove_nan_rows(df_census_temp2)
display(df_census_temp3)    # For debugging only - comment out when not needed.

,State,Label (Grouping),TOTAL NUMBER OF RACES REPORTED!!Total population,TOTAL NUMBER OF RACES REPORTED!!Total population!!One race,TOTAL NUMBER OF RACES REPORTED!!Total population!!Two races,TOTAL NUMBER OF RACES REPORTED!!Total population!!Three races,TOTAL NUMBER OF RACES REPORTED!!Total population!!Four or more races,SEX AND AGE!!Total population,SEX AND AGE!!Total population!!Male,SEX AND AGE!!Total population!!Female,...,OWNER CHARACTERISTICS!!Owner-occupied housing units!!Median selected monthly owner costs with a mortgage (dollars),OWNER CHARACTERISTICS!!Owner-occupied housing units!!Median selected monthly owner costs without a mortgage (dollars),GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed),GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!Less than 30 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!30 percent or more,GROSS RENT!!Occupied units paying rent,GROSS RENT!!Occupied units paying rent!!Median gross rent (dollars),COMPUTERS AND INTERNET USE!!Total households,COMPUTERS AND INTERNET USE!!Total households!!With a computer,COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription
2,United States,Estimate,"333,287,562",87.50%,11.80%,0.70%,0.10%,"333,287,562",49.60%,50.40%,...,"1,775",585,"42,007,061",48.10%,51.90%,"42,971,061","1,300","129,870,928",95.70%,91.00%
3,United States,Margin of Error,*****,±0.1,±0.1,±0.1,±0.1,*****,±0.1,±0.1,...,±3,±1,"±160,112",±0.2,±0.2,"±162,515",±3,"±136,261",±0.1,±0.1
6,Alabama,Estimate,"5,074,296",94.90%,4.70%,0.30%,0.10%,"5,074,296",48.50%,51.50%,...,"1,293",395,"517,883",50.50%,49.50%,"535,960",913,"2,016,448",94.00%,87.40%
7,Alabama,Margin of Error,*****,±0.2,±0.2,±0.1,±0.1,*****,±0.1,±0.1,...,±15,±5,"±11,892",±1.5,±1.5,"±11,920",±14,"±11,475",±0.3,±0.4
10,Alaska,Estimate,"733,583",85.80%,12.80%,1.30%,0.20%,"733,583",52.60%,47.40%,...,"2,019",629,"84,809",56.90%,43.10%,"85,226","1,329","274,574",97.30%,91.60%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,"Spokane County, Washington",Margin of Error,*****,±0.7,±0.7,±0.2,±0.1,*****,±0.2,±0.2,...,±45,±23,"±3,416",±3.8,±3.8,"±3,439",±35,"±2,189",±0.8,±0.9
778,"Dane County, Wisconsin",Estimate,"568,203",90.90%,8.70%,0.40%,0.00%,"568,203",50.20%,49.80%,...,"1,998",856,"103,382",49.60%,50.40%,"105,698","1,258","252,136",97.20%,93.40%
779,"Dane County, Wisconsin",Margin of Error,*****,±0.8,±0.9,±0.2,±0.1,*****,±0.1,±0.1,...,±50,±21,"±3,374",±3.0,±3.0,"±3,410",±36,"±1,877",±0.6,±0.8
782,"Milwaukee County, Wisconsin",Estimate,"918,661",85.50%,13.90%,0.60%,0.10%,"918,661",48.70%,51.30%,...,"1,544",684,"188,551",49.10%,50.90%,"192,905","1,012","388,440",94.40%,90.40%


In [ ]:
# og_string = "!!"
# new_string = " -- "
# df_census_temp4 = col_name_changer(df_census_temp3, og_string, new_string)
# # display(df_census_temp4)     # For debugging only - comment out when not needed.

In [19]:
df_state_only, df_city_state = df_split_state_city(df_census_temp3, 'State')
display(df_state_only)    # For debugging only - comment out when not needed.
# display(df_state_city)    # For debugging only - comment out when not needed.

,State,Label (Grouping),TOTAL NUMBER OF RACES REPORTED!!Total population,TOTAL NUMBER OF RACES REPORTED!!Total population!!One race,TOTAL NUMBER OF RACES REPORTED!!Total population!!Two races,TOTAL NUMBER OF RACES REPORTED!!Total population!!Three races,TOTAL NUMBER OF RACES REPORTED!!Total population!!Four or more races,SEX AND AGE!!Total population,SEX AND AGE!!Total population!!Male,SEX AND AGE!!Total population!!Female,...,OWNER CHARACTERISTICS!!Owner-occupied housing units!!Median selected monthly owner costs with a mortgage (dollars),OWNER CHARACTERISTICS!!Owner-occupied housing units!!Median selected monthly owner costs without a mortgage (dollars),GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed),GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!Less than 30 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!30 percent or more,GROSS RENT!!Occupied units paying rent,GROSS RENT!!Occupied units paying rent!!Median gross rent (dollars),COMPUTERS AND INTERNET USE!!Total households,COMPUTERS AND INTERNET USE!!Total households!!With a computer,COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription
0,United States,Estimate,"333,287,562",87.50%,11.80%,0.70%,0.10%,"333,287,562",49.60%,50.40%,...,"1,775",585,"42,007,061",48.10%,51.90%,"42,971,061","1,300","129,870,928",95.70%,91.00%
1,United States,Margin of Error,*****,±0.1,±0.1,±0.1,±0.1,*****,±0.1,±0.1,...,±3,±1,"±160,112",±0.2,±0.2,"±162,515",±3,"±136,261",±0.1,±0.1
2,Alabama,Estimate,"5,074,296",94.90%,4.70%,0.30%,0.10%,"5,074,296",48.50%,51.50%,...,"1,293",395,"517,883",50.50%,49.50%,"535,960",913,"2,016,448",94.00%,87.40%
3,Alabama,Margin of Error,*****,±0.2,±0.2,±0.1,±0.1,*****,±0.1,±0.1,...,±15,±5,"±11,892",±1.5,±1.5,"±11,920",±14,"±11,475",±0.3,±0.4
4,Alaska,Estimate,"733,583",85.80%,12.80%,1.30%,0.20%,"733,583",52.60%,47.40%,...,"2,019",629,"84,809",56.90%,43.10%,"85,226","1,329","274,574",97.30%,91.60%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,West Virginia,Margin of Error,*****,±0.3,±0.3,±0.1,±0.1,*****,±0.2,±0.2,...,±17,±6,"±6,456",±2.1,±2.1,"±6,256",±17,"±6,960",±0.6,±0.7
100,Wisconsin,Estimate,"5,892,539",92.10%,7.50%,0.40%,0.00%,"5,892,539",50.20%,49.80%,...,"1,545",616,"748,787",54.60%,45.40%,"763,110",992,"2,491,121",94.90%,90.60%
101,Wisconsin,Margin of Error,*****,±0.2,±0.2,±0.1,±0.1,*****,±0.1,±0.1,...,±9,±5,"±11,064",±1.0,±1.0,"±11,107",±9,"±7,766",±0.2,±0.3
102,Wyoming,Estimate,"581,381",91.70%,7.70%,0.60%,0.00%,"581,381",51.20%,48.80%,...,"1,564",484,"58,178",55.90%,44.10%,"59,616",895,"243,321",95.70%,89.50%


In [21]:
# Remove State/location we do not care for.
state_remove = ['Guam', 'District of Columbia', 'Puerto Rico', 'United States', 'Virgin Islands']
df_state_only_updated = remove_rows(df_state_only, 'State', state_remove)
display(df_state_only_updated)

,State,Label (Grouping),TOTAL NUMBER OF RACES REPORTED!!Total population,TOTAL NUMBER OF RACES REPORTED!!Total population!!One race,TOTAL NUMBER OF RACES REPORTED!!Total population!!Two races,TOTAL NUMBER OF RACES REPORTED!!Total population!!Three races,TOTAL NUMBER OF RACES REPORTED!!Total population!!Four or more races,SEX AND AGE!!Total population,SEX AND AGE!!Total population!!Male,SEX AND AGE!!Total population!!Female,...,OWNER CHARACTERISTICS!!Owner-occupied housing units!!Median selected monthly owner costs with a mortgage (dollars),OWNER CHARACTERISTICS!!Owner-occupied housing units!!Median selected monthly owner costs without a mortgage (dollars),GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed),GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!Less than 30 percent,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!30 percent or more,GROSS RENT!!Occupied units paying rent,GROSS RENT!!Occupied units paying rent!!Median gross rent (dollars),COMPUTERS AND INTERNET USE!!Total households,COMPUTERS AND INTERNET USE!!Total households!!With a computer,COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription
2,Alabama,Estimate,"5,074,296",94.90%,4.70%,0.30%,0.10%,"5,074,296",48.50%,51.50%,...,"1,293",395,"517,883",50.50%,49.50%,"535,960",913,"2,016,448",94.00%,87.40%
3,Alabama,Margin of Error,*****,±0.2,±0.2,±0.1,±0.1,*****,±0.1,±0.1,...,±15,±5,"±11,892",±1.5,±1.5,"±11,920",±14,"±11,475",±0.3,±0.4
4,Alaska,Estimate,"733,583",85.80%,12.80%,1.30%,0.20%,"733,583",52.60%,47.40%,...,"2,019",629,"84,809",56.90%,43.10%,"85,226","1,329","274,574",97.30%,91.60%
5,Alaska,Margin of Error,*****,±1.0,±0.9,±0.3,±0.1,*****,±0.3,±0.3,...,±56,±18,"±4,489",±3.3,±3.3,"±4,502",±31,"±3,261",±0.5,±0.6
6,Arizona,Estimate,"7,359,197",80.60%,18.50%,0.80%,0.10%,"7,359,197",50.00%,50.00%,...,"1,616",468,"867,455",47.20%,52.80%,"887,809","1,450","2,850,377",96.40%,91.10%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,West Virginia,Margin of Error,*****,±0.3,±0.3,±0.1,±0.1,*****,±0.2,±0.2,...,±17,±6,"±6,456",±2.1,±2.1,"±6,256",±17,"±6,960",±0.6,±0.7
100,Wisconsin,Estimate,"5,892,539",92.10%,7.50%,0.40%,0.00%,"5,892,539",50.20%,49.80%,...,"1,545",616,"748,787",54.60%,45.40%,"763,110",992,"2,491,121",94.90%,90.60%
101,Wisconsin,Margin of Error,*****,±0.2,±0.2,±0.1,±0.1,*****,±0.1,±0.1,...,±9,±5,"±11,064",±1.0,±1.0,"±11,107",±9,"±7,766",±0.2,±0.3
102,Wyoming,Estimate,"581,381",91.70%,7.70%,0.60%,0.00%,"581,381",51.20%,48.80%,...,"1,564",484,"58,178",55.90%,44.10%,"59,616",895,"243,321",95.70%,89.50%


In [23]:
# Filter out only columns we care about.
final_col_list = [
    'State',
    'Label (Grouping)',
    'SEX AND AGE!!Total population',
    'SEX AND AGE!!Total population!!Male',
    'SEX AND AGE!!Total population!!Female',
    'SEX AND AGE!!Total population!!18 years and over',
    'SEX AND AGE!!Total population!!18 years and over!!Male',
    'SEX AND AGE!!Total population!!18 years and over!!Female',
    'EDUCATIONAL ATTAINMENT!!Population 25 years and over',
    'EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher',
    'EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher!!Male, high school graduate or higher',
    'EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher!!Female, high school graduate or higher',
    'VETERAN STATUS!!Civilian population 18 years and over',
    'VETERAN STATUS!!Civilian population 18 years and over!!Civilian veteran',
    'EMPLOYMENT STATUS!!Population 16 years and over',
    'EMPLOYMENT STATUS!!Population 16 years and over!!In labor force',
    'EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force',
    'EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed',
    'EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed',
    'EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed!!Unemployment Rate',
    'EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces',
    'EMPLOYMENT STATUS!!Population 16 years and over!!Not in labor force',
    'COMMUTING TO WORK!!Workers 16 years and over',
    'COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van - drove alone',
    'COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van - carpooled',
    'COMMUTING TO WORK!!Workers 16 years and over!!Public transportation (excluding taxicab)',
    'COMMUTING TO WORK!!Workers 16 years and over!!Walked',
    'COMMUTING TO WORK!!Workers 16 years and over!!Other means',
    'COMMUTING TO WORK!!Workers 16 years and over!!Worked from home',
    'COMMUTING TO WORK!!Workers 16 years and over!!Mean travel time to work (minutes)',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Households',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Households!!Median household income (dollars)',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Households!!With earnings',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Households!!With earnings!!Mean earnings (dollars)',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Households!!With Social Security income',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Households!!With Social Security income!!Mean Social Security income (dollars)',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Households!!With Supplemental Security Income',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Households!!With Supplemental Security Income!!Mean Supplemental Security Income (dollars)',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Households!!With cash public assistance income',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Households!!With cash public assistance income!!Mean cash public assistance income (dollars)',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Households!!With retirement income',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Households!!With retirement income!!Mean retirement income (dollars)',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Households!!With Food Stamp/SNAP benefits',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Individuals',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Individuals!!Per capita income (dollars)',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Individuals!!With earnings for full-time, year-round workers:!!Male',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Individuals!!With earnings for full-time, year-round workers:!!Female',
    # 'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Individuals!!Mean earnings (dollars) for full-time, year-round workers:!!Male',
    # 'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Individuals!!Mean earnings (dollars) for full-time, year-round workers:!!Female',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Individuals!!Median earnings (dollars) full-time, year-round workers:!!Male',
    'INCOME IN THE PAST 12 MONTHS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Individuals!!Median earnings (dollars) full-time, year-round workers:!!Female',
    'HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population',
    'HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With private health insurance',
    'HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With public coverage',
    'HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!No health insurance coverage',
    'POVERTY RATES FOR FAMILIES AND PEOPLE FOR WHOM POVERTY STATUS IS DETERMINED!!All people!!18 years and over',
    'VEHICLES AVAILABLE!!Occupied housing units',
    'VEHICLES AVAILABLE!!Occupied housing units!!None',
    'VEHICLES AVAILABLE!!Occupied housing units!!1 or more',
    'SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Housing units with a mortgage (excluding units where SMOC cannot be computed)',
    'SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Housing units with a mortgage (excluding units where SMOC cannot be computed)!!Less than 30 percent',
    'SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Housing units with a mortgage (excluding units where SMOC cannot be computed)!!30 percent or more'
]
df_state_only_final = df_state_only_updated[final_col_list]
display(df_state_only_final)
print(df_state_only_final.columns)

,State,Label (Grouping),SEX AND AGE!!Total population,SEX AND AGE!!Total population!!Male,SEX AND AGE!!Total population!!Female,SEX AND AGE!!Total population!!18 years and over,SEX AND AGE!!Total population!!18 years and over!!Male,SEX AND AGE!!Total population!!18 years and over!!Female,EDUCATIONAL ATTAINMENT!!Population 25 years and over,EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,...,HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With private health insurance,HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With public coverage,HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!No health insurance coverage,POVERTY RATES FOR FAMILIES AND PEOPLE FOR WHOM POVERTY STATUS IS DETERMINED!!All people!!18 years and over,VEHICLES AVAILABLE!!Occupied housing units,VEHICLES AVAILABLE!!Occupied housing units!!None,VEHICLES AVAILABLE!!Occupied housing units!!1 or more,SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Housing units with a mortgage (excluding units where SMOC cannot be computed),SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Housing units with a mortgage (excluding units where SMOC cannot be computed)!!Less than 30 percent,SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS!!Housing units with a mortgage (excluding units where SMOC cannot be computed)!!30 percent or more
2,Alabama,Estimate,"5,074,296",48.50%,51.50%,78.10%,47.80%,52.20%,"3,474,924",88.80%,...,67.00%,38.40%,8.80%,14.60%,"2,016,448",5.20%,94.80%,"768,014",77.00%,23.00%
3,Alabama,Margin of Error,*****,±0.1,±0.1,±0.1,±0.1,±0.1,"±5,963",±0.3,...,±0.5,±0.4,±0.3,±0.4,"±11,475",±0.3,±0.3,"±12,780",±0.8,±0.8
4,Alaska,Estimate,"733,583",52.60%,47.40%,76.00%,53.10%,46.90%,"489,218",93.30%,...,66.40%,37.20%,11.00%,10.10%,"274,574",9.10%,90.90%,"110,040",70.90%,29.10%
5,Alaska,Margin of Error,*****,±0.3,±0.3,±0.2,±0.2,±0.2,"±1,732",±0.7,...,±1.2,±1.2,±0.8,±0.8,"±3,261",±0.8,±0.8,"±4,022",±2.3,±2.3
6,Arizona,Estimate,"7,359,197",50.00%,50.00%,78.40%,49.70%,50.30%,"5,053,656",89.20%,...,63.90%,38.30%,10.30%,11.50%,"2,850,377",5.10%,94.90%,"1,180,749",73.70%,26.30%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,West Virginia,Margin of Error,*****,±0.2,±0.2,±0.1,±0.1,±0.1,"±3,566",±0.5,...,±1.0,±0.9,±0.4,±0.7,"±6,960",±0.6,±0.6,"±7,604",±1.3,±1.3
100,Wisconsin,Estimate,"5,892,539",50.20%,49.80%,78.90%,49.80%,50.20%,"4,088,500",93.50%,...,73.30%,35.70%,5.20%,10.20%,"2,491,121",6.50%,93.50%,"1,031,942",76.50%,23.50%
101,Wisconsin,Margin of Error,*****,±0.1,±0.1,±0.1,±0.1,±0.1,"±3,825",±0.2,...,±0.4,±0.4,±0.2,±0.3,"±7,766",±0.3,±0.3,"±11,026",±0.5,±0.5
102,Wyoming,Estimate,"581,381",51.20%,48.80%,77.70%,51.00%,49.00%,"399,796",93.70%,...,71.00%,31.20%,11.50%,11.10%,"243,321",4.10%,95.90%,"101,037",73.70%,26.30%


Index(['State', 'Label (Grouping)', 'SEX AND AGE!!Total population',
       'SEX AND AGE!!Total population!!Male',
       'SEX AND AGE!!Total population!!Female',
       'SEX AND AGE!!Total population!!18 years and over',
       'SEX AND AGE!!Total population!!18 years and over!!Male',
       'SEX AND AGE!!Total population!!18 years and over!!Female',
       'EDUCATIONAL ATTAINMENT!!Population 25 years and over',
       'EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher',
       'EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher!!Male, high school graduate or higher',
       'EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher!!Female, high school graduate or higher',
       'VETERAN STATUS!!Civilian population 18 years and over',
       'VETERAN STATUS!!Civilian population 18 years and over!!Civilian veteran',
       'EMPLOYMENT STATUS!!Population 16 years and over',
       'EMPLOYME

In [ ]:
# save_df_to_csv(df_state_only, './data/processed/US_Census_Data_2022_state_only.csv')

In [ ]:
# save_df_to_csv(df_city_state, './data/processed/US_Census_Data_2022_city_state.csv') # Do not need city_state level since chronic disease dataset does not have that level.

In [32]:
df_state_only_estimate, df_state_only_moe = df_split(df_state_only_final, 'Label (Grouping)', 'Estimate', 'Margin of Error')
# display(df_state_only_estimate)         # For debugging only - comment out when not needed.
# display(df_state_only_moe)         # For debugging only - comment out when not needed.

In [34]:
save_df_to_csv(df_state_only_estimate, './data/processed/US_Census_Data_2022_state_only_estimate.csv')
save_df_to_csv(df_state_only_moe, './data/processed/US_Census_Data_2022_state_only_moe.csv')

Dataframe:               State Label (Grouping) SEX AND AGE!!Total population  \
2           Alabama         Estimate                     5,074,296   
4            Alaska         Estimate                       733,583   
6           Arizona         Estimate                     7,359,197   
8          Arkansas         Estimate                     3,045,637   
10       California         Estimate                    39,029,342   
12         Colorado         Estimate                     5,839,926   
14      Connecticut         Estimate                     3,626,205   
16         Delaware         Estimate                     1,018,396   
20          Florida         Estimate                    22,244,823   
22          Georgia         Estimate                    10,912,876   
24           Hawaii         Estimate                     1,440,196   
26            Idaho         Estimate                     1,939,033   
28         Illinois         Estimate                    12,582,032   
30       